# Basic Financial Calculations

In this post I'll cover:

- Net present value (NPV)
- Internal rate of return (IRR)
- Payment schedules and loan tables
- Future value
- Pension and accumulation problems
- Continuously compounded interest

## Net present value 

Present value allows to answer a simple question "should I put my money in the bank or invest". Let's say I have \$100 and bank gives 12% annual interest rate (1% per month). Alternatively I can lend this money to a friend and he promises to pay \$10 for 12 month. Moral aspect aside NPV gives an answer which alternative is more profitable.  

Second alternative can be described as a series of cash flows: 
    - -\$100 at month=0
    - +\$10 at month=1
    ...
    - +\$10 at month=12

In [1]:
import pandas as pd
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta

df = pd.DataFrame(data={'cash flow': [-100] + [10]*12, 'date':[date.today() + relativedelta(months=i) for i in range(13) ]})

df

,cash flow,date
0,-100,2018-12-13
1,10,2019-01-13
2,10,2019-02-13
3,10,2019-03-13
4,10,2019-04-13
5,10,2019-05-13
6,10,2019-06-13
7,10,2019-07-13
8,10,2019-08-13
9,10,2019-09-13


In [17]:
help(relativedelta)

Help on class relativedelta in module dateutil.relativedelta:

class relativedelta(builtins.object)
 |  The relativedelta type is based on the specification of the excellent
 |  work done by M.-A. Lemburg in his
 |  `mx.DateTime <https://www.egenix.com/products/python/mxBase/mxDateTime/>`_ extension.
 |  However, notice that this type does *NOT* implement the same algorithm as
 |  his work. Do *NOT* expect it to behave like mx.DateTime's counterpart.
 |  
 |  There are two different ways to build a relativedelta instance. The
 |  first one is passing it two date/datetime classes::
 |  
 |      relativedelta(datetime1, datetime2)
 |  
 |  The second one is passing it any number of the following keyword arguments::
 |  
 |      relativedelta(arg1=x,arg2=y,arg3=z...)
 |  
 |      year, month, day, hour, minute, second, microsecond:
 |          Absolute information (argument is singular); adding or subtracting a
 |          relativedelta with absolute information does not perform an arithm

At each period value should be adjusted by the interest rate (also called the discount rate) using formula 
![npv](http://www.sciweavers.org/tex2img.php?eq=%5Cfrac%7BCF_%7Bt%7D%20%7D%7B%20%281%20%2B%20r%29%5E%7Bt%7D%20%7D&bc=White&fc=Black&im=jpg&fs=12&ff=arev&edit=0)
This basic concept shows that \$100 now is not the same as \$100 promised in a year. \$100 now is $110 in a year.
To calcualte discounted value I'll conver annual discount (interest) rate to daily and use the number of days elapsed as *t*  

In [2]:
def _annual_to_daily_ir(annual_interest_rate):
    return (1 + annual_interest_rate)**(1/365) - 1

def _daily_to_annual_ir(daily_interest_rate):
    return (1 + daily_interest_rate)**365 - 1

def _calculate_discounted(cf, annual_interest_rate, days_passed):
    daily_interest_rate = convert_ir(annual_interest_rate, from_period='year', to_period='day')
    return cf / (1 + daily_interest_rate)**days_passed
_daily_to_annual_ir(0.00026), _annual_to_daily_ir(0.1)

(0.09953533203843645, 0.00026115787606784124)

In [3]:
days_in_year = 365
month_in_year = 12
quaters_in_year = 4
month_in_quater = 3
days_in_week = 7
1/(days_in_week/days_in_year)

52.14285714285714

In [4]:
def convert_ir(r, from_period='year', to_period='day'):
    days_in_year = 365
    month_in_year = 12
    quaters_in_year = 4
    month_in_quater = 3
    days_in_week = 7
    powers = {'year': {'day': 1/days_in_year,
                       'week': days_in_week/days_in_year,
                       'month': 1/month_in_year,
                       'quater': 1/quaters_in_year,
                       'year': 1},
              'quater': {'day': 1/(days_in_year/quaters_in_year),
                         'week': 1/(days_in_year/quaters_in_year/days_in_week),
                         'month': 1/month_in_quater,
                         'quater': 1,
                         'year': quaters_in_year},
              'month': {'day': 1/(days_in_year/month_in_year),
                        'week': 1/(days_in_year/month_in_year/days_in_week),
                        'month': 1,
                        'quater': month_in_quater,
                        'year': month_in_year},
              'week': {'day': 1/days_in_week,
                       'week': 1,
                       'month': days_in_year/month_in_year/days_in_week,
                       'quater':days_in_year/quaters_in_year/days_in_week,
                       'year': days_in_year/days_in_week},
              'day': {'day': 1,
                      'week': days_in_week,
                      'month': days_in_year/month_in_year,
                      'quater': days_in_year/quaters_in_year,
                      'year': days_in_year}}
    return (1 + r)**(powers[from_period][to_period]) - 1
convert_ir(0.1), convert_ir(0.1, 'year', 'quater')

(0.00026115787606784124, 0.02411368908444511)

To calculate NPV I'll sum up all discounted values at each *t*:

In [5]:
#def npv(df, cash_flow_column_name, annual_discount_rate):
cash_flow_column_name = 'cash flow'
date_column_name = 'date'
annual_discount_rate = 0.1
def npv(dataframe, annual_discount_rate, cash_flow_column_name='cash flow', date_column_name='date'):
    date_start = dataframe[date_column_name].min()
    mapper = lambda x: _calculate_discounted(x[cash_flow_column_name], 
                                             annual_discount_rate, 
                                             (x[date_column_name] - date_start).days)
    return dataframe[[cash_flow_column_name, date_column_name]].apply(mapper, axis=1).sum()
npv(df, 0.1)

14.011086147172053

The calcualted NPV of lending alternative is ~\$14 which means that lending to a friend is profitable. This value represents your wealth increment.  Companies use NPV
to calculate if it's, for example, vialble to invest in a factory that will generate cash flows in a future or it's better to put money elsewhere.

`npv` function can also operate with time-dated cash flows. 

In [6]:
# this is a test from a book
df1 = pd.DataFrame(data={'cash flow': [-250] + [100]*5, 'date':[date.today() + relativedelta(years=i) for i in range(6) ]})
npv(df1, 0.1)

129.00343883282264

## Internal rate of returns

So I decided to lend to a friend. What is the effective interest rate for this alternative? Interal rate of returns 
(IRR) answers that. The solution of the following equation for *r* gives IRR:
    ![irr](http://www.sciweavers.org/tex2img.php?eq=CF_%7B0%7D%20%2B%20%5Csum_%7Bt%3D1%7D%5E%7BN%7D%20%5Cfrac%7BCF_%7Bt%7D%20%7D%7B%281%20%2B%20r%29%5E%7Bt%7D%20%7D%20%3D%200&bc=White&fc=Black&im=jpg&fs=12&ff=arev&edit=0)
    
Effectively we are finding interest rate when NPV is 0.

In [7]:
from scipy.optimize import fsolve

In [8]:
from functools import partial
def irr(dataframe, guess=0, cash_flow_column_name='cash flow', date_column_name='date'):
    f = partial(npv, dataframe, cash_flow_column_name=cash_flow_column_name)
    result = fsolve(f, guess)
    return list(result)
    
df2 = pd.DataFrame(data={'cash flow': [-800, 200, 250, 300, 350, 400], 'date':[date.today() + relativedelta(years=i) for i in range(6) ]})
irr(df, 0.0), irr(df2, 0.0)

([0.41354120272404077], [0.22142234649016324])

Lending to a friend has an effetive interest rate of 41%

Sometimes series of cash flows can have more than one IRR. *Guess* parameter is the initial gussed value to solve the equation. We can play with it to find both solutions. 

In [9]:
df3 = pd.DataFrame(data={'cash flow': [-145, 100, 100, 100, 100, -275], 'date':[date.today() + relativedelta(years=i) for i in range(6) ]})
irr(df3, 0.1), irr(df3, 0.4)

([0.08793680470699422], [0.26585705483081506])

In [10]:
100/12

8.333333333333334

## Flat payment schedules

Let's look at the lending problems from the friends point. He knows that with the cash flow he proposed he will be paying 41% annual interest rate. He wants to lower the interest rate to say 30% and calculates new flat payment scheme. Flat here means that he will pay contant amount each month for a *term* amount pf months.

First let's calculate monthly payment:

In [11]:
def pmt(principal, annual_interest_rate, term, period='year'):
    periodic_interest = convert_ir(annual_interest_rate, from_period='year', to_period=period)
    payment = (principal*periodic_interest)/(1 - (1 + periodic_interest)**(-term))
    return payment

In [12]:
pmt(10000, 0.07, 6, period='year')

2097.9579975832817

In [13]:
pmt(100, 0.3, 12, period='month')

9.57859525723352

So he will be paying ~$9.6 each month. Now he wants to know how fast he's payin the interest and the principal. For that a loan table is used.

In [89]:
from collections import defaultdict

def flat_payments(principal, annual_interest_rate, term, period='year'):
    _pmt = pmt(principal, annual_interest_rate, term, period=period)
    data = defaultdict(list)
    current_principal = principal
    principal_col_name = 'principal at the beginning of {}'.format(period)
    payment_col_name = 'payment at the end of {}'.format(period)
    interest_col_name = 'interest'
    return_principal_col_name = 'return of principal'
    periodic_interest = convert_ir(annual_interest_rate, from_period='year', to_period=period)
    for _t in range(1, term + 1):
        data[period].append(_t)
        data[principal_col_name].append(current_principal)
        data[payment_col_name] = _pmt
        current_interest = current_principal * periodic_interest
        data[interest_col_name].append(current_interest)
        current_return_of_principal = _pmt - current_interest
        data[return_principal_col_name].append(current_return_of_principal)
        current_principal -= current_return_of_principal  
    assert round(current_principal) == 0
    print(data)
    return pd.DataFrame.from_dict(data)

flat_payments(10000, 0.1, 2, period='year')

defaultdict(<class 'list'>, {'year': [1, 2], 'principal at the beginning of year': [10000, 5238.095238095237], 'payment at the end of year': 5761.904761904764, 'interest': [1000.0000000000009, 523.8095238095241], 'return of principal': [4761.904761904763, 5238.09523809524]})


,year,principal at the beginning of year,payment at the end of year,interest,return of principal
0,1,10000.000000,5761.904762,1000.000000,4761.904762
1,2,5238.095238,5761.904762,523.809524,5238.095238


In [15]:
flat_payments(100, 0.3, 12, period='month')

,month,principal at the beginning of month,payment at the end of month,interest,return of principal
0,1,100.000000,9.578595,2.210445,7.368150
1,2,92.631850,9.578595,2.047576,7.531019
2,3,85.100831,9.578595,1.881107,7.697488
3,4,77.403343,9.578595,1.710958,7.867637
4,5,69.535706,9.578595,1.537049,8.041547
5,6,61.494159,9.578595,1.359295,8.219301
6,7,53.274858,9.578595,1.177611,8.400984
7,8,44.873875,9.578595,0.991912,8.586683
8,9,36.287192,9.578595,0.802108,8.776487
9,10,27.510705,9.578595,0.608109,8.970486


Last two columns - *interest* and *return of principal* - show how the payment is split between repaying interest and principal. At the end principal value should be 0 (see assert in function code). It's interesting to know how much interest is repayed because that amount sometimes is tax deductable.

## Future value

Let's imagine we put \$10000 today for 10 years with annual interest rate 10%. How much money we will have in 10 years? Using this formula with r=0.1 ant t=1- will give us \$2593.74
    ![npv](http://www.sciweavers.org/tex2img.php?eq=%7BDeposit%7D%2A%7B%281%20%2B%20r%29%5E%7Bt%7D%20%7D%20&bc=White&fc=Black&im=jpg&fs=12&ff=arev&edit=0)
    
Slightly more complex problem if we want to deposit some money during this 10 years. *Future value* will tell how much money we'll have in the end of the period.

In [16]:
def fv(deposits, annual_interest_rate, period='year'):
    periodic_interest = convert_ir(annual_interest_rate, from_period='year', to_period=period)
    future_value = 0
    number_of_deposits = len(deposits)
    for n, deposit in enumerate(deposits):
        future_value += deposit * (1 + periodic_interest)**(number_of_deposits - n)
    return future_value
fv([1000 for _ in range(10)], 0.1, period='year')

17531.16706110001

## Retirement problem
Consider a following exersize. Now I'm 31 and planning to retire at 55 (24 more years working). After retiring I'm planning to live at least 25 more years and will need, say, \$50000 a year. To support my retirement I need to deposit money into bank account with know interest rate (say 5%). What's the minimum amount do I need to deposit each year?

This problem implies a number of cash flows: 24 terms with CF=*x* at each time, and 25 terms with CF=-50000. Present value of this cash flows should be 0. Solving this equation for *x* will give the minimum annual deposit. Minimum deposit is used here not in mathematical sence, but more in common sence - I'll live more that 25 years after retirement, prices are also going up so I'll need more that 50k yearly.


In [48]:
from functools import partial
def get_retirement_cf_dataframe(deposit, terms_of_deposit, withdrawal, terms_of_withdrawal, period='year'):
    cash_flows = [deposit]*terms_of_deposit + [-withdrawal]*terms_of_withdrawal
    terms = terms_of_deposit + terms_of_withdrawal
    dates = [date.today() + relativedelta(**{period + 's':i}) for i in range(terms)]
    df = pd.DataFrame(data={'cash flow': cash_flows, 'date': dates})
    return df


def retirement_problem(terms_of_deposit, withdrawal, terms_of_withdrawal, annual_discount_rate, period='year'):
    retirement_dataframe_by_deposit = partial(get_retirement_cf_dataframe, 
                                              terms_of_deposit=terms_of_deposit, 
                                              withdrawal=withdrawal, 
                                              terms_of_withdrawal=terms_of_withdrawal, 
                                              period=period)
    f = lambda deposit: npv(retirement_dataframe_by_deposit(deposit), annual_discount_rate)
    result = fsolve(f, withdrawal)
    return list(result)

retirement_problem(24, 50000, 25, 0.05)

[15822.327630785972]

## Countinous compounding
I'm using `convert_ir(r, from_period='year', to_period='day')` function to convert from annual interest rate to daily interest rate in some examples above.
So if I have annual rate of 20%, quaterly rate is:

In [49]:
convert_ir(0.2, from_period='year', to_period='quater')

0.04663513939210562

And respective annual is:

In [54]:
4 * convert_ir(0.2, from_period='year', to_period='quater'),  2*((1 + 0.2)**(1/2) - 1)

(0.18654055756842247, 0.1908902300206643)

So if a bank offers 20% annual interest rate paid quaterly the effective annual interest rate is 18.6%. if the number of interest payments increaces - for example to every millisecond. This is continuous compounding. Let's calcualte annual interest rate continously compounded given intial and end anounts.

In [76]:
df = pd.DataFrame(data={'amount': [1000, 1500], 'date': [date.today(), date.today() + relativedelta(years=1, months=9)]})
df

,amount,date
0,1000,2018-12-13
1,1500,2020-09-13


In [81]:
from math import log
help(relativedelta)

Help on class relativedelta in module dateutil.relativedelta:

class relativedelta(builtins.object)
 |  The relativedelta type is based on the specification of the excellent
 |  work done by M.-A. Lemburg in his
 |  `mx.DateTime <https://www.egenix.com/products/python/mxBase/mxDateTime/>`_ extension.
 |  However, notice that this type does *NOT* implement the same algorithm as
 |  his work. Do *NOT* expect it to behave like mx.DateTime's counterpart.
 |  
 |  There are two different ways to build a relativedelta instance. The
 |  first one is passing it two date/datetime classes::
 |  
 |      relativedelta(datetime1, datetime2)
 |  
 |  The second one is passing it any number of the following keyword arguments::
 |  
 |      relativedelta(arg1=x,arg2=y,arg3=z...)
 |  
 |      year, month, day, hour, minute, second, microsecond:
 |          Absolute information (argument is singular); adding or subtracting a
 |          relativedelta with absolute information does not perform an arithm

In [85]:
def get_annual_rate_cc(dataframe, amount_column_name='amount', date_column_name='date'):
    amount_first = dataframe[amount_column_name].iloc[0]
    amount_last = dataframe[amount_column_name].iloc[-1]
    date_first = dataframe[date_column_name].iloc[0]
    date_last = dataframe[date_column_name].iloc[-1]
    delta = relativedelta(date_last, date_first)
    t = delta.years + delta.months/12 + delta.days/365
    r = log(amount_last/amount_first)/t
    return r
get_annual_rate_cc(df)

0.23169434749037965